# H_22082024
# Analisis del volumen profile para encontrar zonas de alta probabilidad y como se comporta el precio en esas zonas.

In [3]:
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
import pytz
from datetime import datetime
import matplotlib.pyplot as plt
import json
import plotly.express as px
import plotly.graph_objects as go

In [4]:
# Variables

STD_MULTIPLIER = 1
ticker = "GC_Z"
start_dt = "2021-05-25"
end_dt = "2024-06-28"
minute_data = "GC_V_M1_20091028_20240628.csv"
daily_data = "GC_V_D1_20091028_20240628.csv"

## Recopilación de datos, manipulación y limpieza

In [5]:
def data_from_mt5():
    # connect to MetaTrader 5
    if not mt5.initialize():
        print("initialize() failed")
        mt5.shutdown()

    # set time zone to UTC
    timezone = pytz.timezone("Etc/UTC")
    # create 'datetime' objects in UTC time zone to avoid the implementation of a local time zone offset
    utc_from = datetime.strptime(start_dt, "%Y-%m-%d")
    utc_to = datetime.strptime(end_dt, "%Y-%m-%d")#datetime(2024, 8, 1, tzinfo=timezone)
    # request AUDUSD ticks within 11.01.2020 - 11.01.2020
    ohlcv = mt5.copy_rates_range(ticker, mt5.TIMEFRAME_M1, utc_from, utc_to)
    daily_ohlcv = mt5.copy_rates_range(ticker, mt5.TIMEFRAME_D1, utc_from, utc_to)
    print(ohlcv)

    mt5.shutdown()

    df = pd.DataFrame(ohlcv)
    df['time']=pd.to_datetime(df['time'], unit='s')

    # adaptamos el dataframe D1 para luego hacer los analisis de las sesiones
    daily_df = pd.DataFrame(daily_ohlcv)
    daily_df['time']=pd.to_datetime(daily_df['time'], unit='s')

    df = df.set_index('time')
    del df['real_volume']
    del df['spread']

    daily_df = daily_df.set_index('time')
    del daily_df['real_volume']
    del daily_df['spread']

    return df, daily_df

def data_from_csv():
    df = pd.read_csv('C:/Users/iamfr/AlgoTrading/DATA/'+minute_data, sep='\t')
    df['<DATE>'] = pd.to_datetime(df['<DATE>'] + ' ' + df['<TIME>'])
    del df['<TIME>']
    del df['<VOL>']
    del df['<SPREAD>']
    df.columns = ['time', 'open','high', 'low', 'close', 'tick_volume']
    df = df.set_index('time')

    daily_df = pd.read_csv('C:/Users/iamfr/AlgoTrading/DATA/'+daily_data, sep='\t')
    #df['<DATE>'] = pd.to_datetime(df['<DATE>'] + ' ' + df['<TIME>'])
    #del df['<TIME>']
    del daily_df['<VOL>']
    del daily_df['<SPREAD>']
    daily_df.columns = ['time', 'open','high', 'low', 'close', 'tick_volume']
    daily_df = daily_df.set_index('time')
    #daily_df = df.resample('1B').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'tick_volume': 'sum'})

    return df, daily_df

df, daily_df = data_from_csv()

df

,open,high,low,close,tick_volume
time,,,,,
2009-10-28 02:36:00,1372.6,1372.6,1372.6,1372.6,1
2009-10-28 02:58:00,1373.1,1373.1,1373.1,1373.1,2
2009-10-28 02:59:00,1372.9,1372.9,1372.9,1372.9,1
2009-10-28 03:00:00,1372.9,1372.9,1372.9,1372.9,1
2009-10-28 03:04:00,1373.1,1373.1,1373.1,1373.1,3
...,...,...,...,...,...
2024-06-28 21:55:00,2359.4,2359.6,2359.1,2359.3,130
2024-06-28 21:56:00,2359.2,2359.4,2359.1,2359.1,123
2024-06-28 21:57:00,2359.0,2359.2,2358.8,2359.1,107


In [6]:
def create_market_profile(data, getPOC=True):
    profile = data.groupby('close')['tick_volume'].sum().reset_index()
    total_volume = profile['tick_volume'].sum()
    profile['volume_cumsum'] = profile['tick_volume'].cumsum()

    value_area_cutoff = total_volume * 0.70
    value_area_df = profile[profile['volume_cumsum'] <= value_area_cutoff]
    POC = 0
    if getPOC:
        POC = profile.loc[profile['tick_volume'].idxmax(), 'close']
    else:
        POC = profile.loc[profile['tick_volume'].idxmin(), 'close']

    return profile, value_area_df, POC

def plot_market_profile(profile, value_area_df, POC):
    plt.figure(figsize=(10, 6))
    plt.barh(profile['close'], profile['tick_volume'], color='blue', edgecolor='black')
    plt.barh(value_area_df['close'], value_area_df['tick_volume'], color='green', edgecolor='black')
    plt.axhline(POC, color='red', linestyle='--', label=f'POC: {POC}')

    plt.xlabel('Volume')
    plt.ylabel('Price')
    plt.title(f'Market Profile')
    plt.legend()
    plt.show()

In [7]:
def get_no_fair_range_zone(df, threshold):
    # Crear una máscara booleana para identificar dónde 'tick_volume' es inferior al umbral
    df.sort_values(by=['close'])
    mask = df['tick_volume'] < threshold

    # Encontrar los índices donde empieza y termina cada zona
    rangos = []
    inicio = None
    rsize = 0
    rango_max = []

    for i in range(len(df)):
        if mask[i]:
            if inicio is None:  # Se inicia una nueva zona
                inicio = i
        else:
            if inicio is not None:  # Se cierra la zona actual
                rangos.append([inicio, i - 1])
                inicio = None

    # Si la última zona no se cierra explícitamente en el bucle
    if inicio is not None:
        rangos.append([inicio, len(df) - 1])
                 
    for rango in rangos:
        if rsize <= (rango[1] - rango[0]):  
              rsize = rango[1] - rango[0]
              rango_max = rango

    if len(rangos) < 1:
        return False
    return [df.loc[rango_max[0], 'close'], df.loc[rango_max[1], 'close']]

def get_max_vol_zone(df, threshold):
    # Crear una máscara booleana para identificar dónde 'tick_volume' es inferior al umbral
    df.sort_values(by=['close'])
    mask = df['tick_volume'] > threshold

    # Encontrar los índices donde empieza y termina cada zona
    rangos = []
    inicio = None
    rsize = 0
    rango_max = []

    for i in range(len(df)):
        if mask[i]:
            if inicio is None:  # Se inicia una nueva zona
                inicio = i
        else:
            if inicio is not None:  # Se cierra la zona actual
                rangos.append([inicio, i - 1])
                inicio = None

    # Si la última zona no se cierra explícitamente en el bucle
    if inicio is not None:
        rangos.append([inicio, len(df) - 1])
                 
    for rango in rangos:
        if rsize <= (rango[1] - rango[0]):  
              rsize = rango[1] - rango[0]
              rango_max = rango

    if len(rangos) < 1:
        return False
    return [df.loc[rango_max[0], 'close'], df.loc[rango_max[1], 'close']]

In [8]:
# iteramos cada dia, y cada zona
def get_no_fair_zone_by_day(df):
    out = []

    for index1, day in df.groupby(df.index.date):

        profile, value_area_df, MIN = create_market_profile(day, False)
        mean = profile['tick_volume'].mean()
        stddev = profile['tick_volume'].std()
        threshold = mean + stddev * STD_MULTIPLIER

        no_fair_value_zone = get_no_fair_range_zone(profile, threshold)

        if no_fair_value_zone == False:
            output = {
                "time": index1,
                "min_zone_high": np.nan,
                "min_zone_low": np.nan,
                "MIN": np.nan,
            }
            out.append(output)
        else:
            output = {
                "time": index1,
                "min_zone_high": no_fair_value_zone[1],
                "min_zone_low": no_fair_value_zone[0],
                "MIN": MIN,
            }
            out.append(output)

    return out

# iteramos cada dia, y cada zona
def get_max_vol_zone_by_day(df):
    out = []

    for index1, day in df.groupby(df.index.date):

        profile, value_area_df, POC = create_market_profile(day)
        mean = profile['tick_volume'].mean()
        stddev = profile['tick_volume'].std()
        threshold = mean + stddev * STD_MULTIPLIER

        max_value_zone = get_max_vol_zone(profile, threshold)

        if max_value_zone == False:
            output = {
                "time": index1,
                "max_zone_high": np.nan,
                "max_zone_low": np.nan,
                "POC": np.nan,
            }
            out.append(output)
        else:
            output = {
                "time": index1,
                "max_zone_high": max_value_zone[1],
                "max_zone_low": max_value_zone[0],
                "POC": POC,
            }
            out.append(output)

    return out

## Analisis de datos

In [9]:
#zones = get_no_fair_zone_by_day(df)
max_zones = get_max_vol_zone_by_day(df)
df_max_zones = pd.DataFrame(max_zones)
#df_zones['time'].astype('datetime64[ns]')
df_max_zones = df_max_zones.set_index('time')
df_max_zones.index.astype('datetime64[ns]')

#zones = get_no_fair_zone_by_day(df)
min_zones = get_no_fair_zone_by_day(df)
df_min_zones = pd.DataFrame(min_zones)
#df_zones['time'].astype('datetime64[ns]')
df_min_zones = df_min_zones.set_index('time')
df_min_zones.index.astype('datetime64[ns]')

main_df_2 = pd.merge(df_max_zones, df_min_zones, on=daily_df.index)
main_df_2 = main_df_2.set_index('key_0')
main_df = pd.merge(daily_df, main_df_2, on=daily_df.index)
#daily_df.join(df_zones, on=daily_df.index , how='inner')
main_df = main_df.set_index('key_0')
main_df.dropna()
main_df

#main_df.to_csv( TICKER+'_H22082024_Output_Data.csv', index=True)

,open,high,low,close,tick_volume,max_zone_high,max_zone_low,POC,min_zone_high,min_zone_low,MIN
key_0,,,,,,,,,,,
2009.10.28,1372.6,1376.8,1357.7,1358.3,890,1371.8,1371.7,1371.7,1376.8,1373.0,1359.2
2009.10.29,1358.9,1384.8,1357.2,1384.2,920,1380.4,1380.4,1368.1,1377.2,1369.4,1358.1
2009.10.30,1383.7,1385.9,1368.6,1370.4,790,1384.7,1384.5,1378.7,1378.6,1373.0,1369.7
2009.10.31,1371.5,1383.5,1371.5,1382.0,512,1374.8,1374.8,1374.8,1383.5,1375.1,1372.3
2009.11.02,1379.2,1405.2,1379.2,1403.3,1144,1402.3,1401.6,1400.5,1390.5,1379.2,1379.8
...,...,...,...,...,...,...,...,...,...,...,...
2024.06.24,2354.2,2370.6,2353.0,2370.4,134044,2360.5,2359.9,2361.3,2358.4,2353.0,2354.6
2024.06.25,2369.5,2372.7,2350.5,2354.6,160721,2362.0,2361.3,2361.1,2372.6,2365.0,2367.7
2024.06.26,2354.5,2358.6,2327.6,2332.4,197254,2351.7,2351.2,2333.2,2347.1,2337.8,2352.3


### SETUP_1. Cierre de rango en la siguiente sesión

Analizaremos si en la siguiente sesión, el precio cierra el rango del dia anterior.

In [10]:
def num1_next_session_close_zone(df):
    results = []

    for i in range(len(df.index)-1):
        if i == 0: pass
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        zone_high = df.iloc[i]['max_zone_high']
        zone_low = df.iloc[i]['max_zone_low']

        # Comprobar si se cumple la condición
        if zone_high <= high and zone_high >= low and zone_low >= low and zone_low <= high:
            results.append(True)  # Cumple la condición
        else:
            results.append(False)  # No cumple la condición

    # Agregar un 0 adicional al final para igualar el tamaño de la columna con el dataframe original
    results.append(False)

    # Crear una nueva columna en el dataframe con los resultados
    df['SETUP_1'] = results

    return df['SETUP_1'].value_counts(normalize=True).mul(100).astype(str)+'%'

print("Porcentaje de cierre de la zona: ", num1_next_session_close_zone(main_df))

Porcentaje de cierre de la zona:  SETUP_1
True      68.78547105561861%
False    31.214528944381385%
Name: proportion, dtype: object


### SETUP_2. Cierre de medio rango en la siguiente sesion

Analizaremos si en la siguiente sesión, el precio cierra la midat del rango.

In [11]:
def num2_next_session_close_half_zone(df):
    results = []

    for i in range(len(df.index) - 1):
        if i == 0: pass
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        zone_high = df.iloc[i]['max_zone_high']
        zone_low = df.iloc[i]['max_zone_low']
        zone_mid = ((zone_high - zone_low) / 2) + zone_low

        if apertura >= zone_high:
            # Mitad superior
            if zone_high <= high and zone_high >= low and zone_mid >= low and zone_mid <= high:
                results.append(True)
            else:
                results.append(False)
        elif apertura <= zone_low:
            # Mitad inferior
            if zone_mid <= high and zone_mid >= low and zone_low >= low and zone_low <= high:
                results.append(True)
            else:
                results.append(False)
        else:
            results.append(False)
    # Agregar un 0 adicional al final para igualar el tamaño de la columna con el dataframe original
    results.append(False)

    # Crear una nueva columna en el dataframe con los resultados
    df['SETUP_2'] = results

    return df['SETUP_2'].value_counts(normalize=True).mul(100).astype(str)+'%'

print("Porcentaje de cierre de media zona: ", num2_next_session_close_half_zone(main_df))

Porcentaje de cierre de media zona:  SETUP_2
True     66.22020431328036%
False    33.77979568671964%
Name: proportion, dtype: object


### SETUP_3. Desde apertura a extremo de zona

Analisis de la cantidad de veces que el precio recorre el rango entre la apertura de la sesion y un valor extremo.

Depende de si la apertura es por debajo o per encima de la zona de alta capitalización, buscaremos largos o cortos.

In [12]:
def num3_from_open_to_range_max_volume(df):
    results = []

    for i in range(len(df.index) - 1):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        zone_high = df.iloc[i]['max_zone_high']
        zone_low = df.iloc[i]['max_zone_low']

        if apertura >= zone_high:
            # Bajista
            if zone_low <= high and zone_low >= low:
                results.append(True)
            else:
                results.append(False)
        elif apertura <= zone_low:
            # Alcista
            if zone_high <= high and zone_high >= low:
                results.append(True)
            else:
                results.append(False)
        else: 
            results.append(False)

    # Agregar un 0 adicional al final para igualar el tamaño de la columna con el dataframe original
    results.append(False)

    # Crear una nueva columna en el dataframe con los resultados
    df['SETUP_3'] = results

    return df['SETUP_3'].value_counts(normalize=True).mul(100).astype(str)+'%'

print("Porcentaje de Apertura-Extremo: ", num3_from_open_to_range_max_volume(main_df))

Porcentaje de Apertura-Extremo:  SETUP_3
True      65.22133938706017%
False    34.778660612939845%
Name: proportion, dtype: object


### SETUP_4. Recorrido a POC anterior session desde apertura 

Analisis de la cantidad de veces que se testea el valor POC de la anterior sesion.

In [13]:
def num4_POC_test(df):
    results = []

    for i in range(len(df.index) - 1):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        poc = df.iloc[i]['POC']

        if apertura >= poc:
            # Bajista
            if poc <= high and poc >= low:
                results.append(True)
            else:
                results.append(False)

        elif apertura <= poc:
            # Alcista
            if poc <= high and poc >= low:
                results.append(True)
            else:
                results.append(False)
        else: 
            results.append(False)

    # Agregar un 0 adicional al final para igualar el tamaño de la columna con el dataframe original
    results.append(False)

    # Crear una nueva columna en el dataframe con los resultados
    df['SETUP_4'] = results

    return df['SETUP_4'].value_counts(normalize=True).mul(100).astype(str)+'%'

print("Porcentaje de test POC: ", num4_POC_test(main_df))

Porcentaje de test POC:  SETUP_4
True      71.19182746878548%
False    28.808172531214527%
Name: proportion, dtype: object


### Nivel de mayor probabilidad estadistica: SETUP_4

A continuación, sacaremos más estadisticas que complementarán el estudio.

In [14]:
def cummulative_not_setup4_true(df):
    results = []
    tmp = 0

    for index, row in df.iterrows():
        if row['SETUP_4'] == False:
            tmp = tmp + 1
        else:
            results.append(tmp)
            tmp = 0

    nparr = np.array(results)

    return nparr.max()

print("Maximos fallos consecutivos de SETUP 4: ", cummulative_not_setup4_true(main_df))

Maximos fallos consecutivos de SETUP 4:  6


In [15]:
def setup4_range_stddev(df):
    df['SETUP_4_range'] = abs(df['POC'] - df['open'])

    true_df = df.query('SETUP_4 == True')
    tmean = true_df['SETUP_4_range'].mean()
    tstddev = true_df['SETUP_4_range'].std()

    false_df = df.query('SETUP_4 == False')
    fmean = false_df['SETUP_4_range'].mean()
    fstddev = false_df['SETUP_4_range'].std()

    return tmean, fmean, tstddev, fstddev

print("Datos de rango MEAN, STDDEV\n", setup4_range_stddev(main_df))

Datos de rango MEAN, STDDEV
 (9.345025510204083, 10.414184397163124, 11.037461257197403, 11.001165864544541)


In [16]:
main_df

,open,high,low,close,tick_volume,max_zone_high,max_zone_low,POC,min_zone_high,min_zone_low,MIN,SETUP_1,SETUP_2,SETUP_3,SETUP_4,SETUP_4_range
key_0,,,,,,,,,,,,,,,,
2009.10.28,1372.6,1376.8,1357.7,1358.3,890,1371.8,1371.7,1371.7,1376.8,1373.0,1359.2,True,True,True,True,0.9
2009.10.29,1358.9,1384.8,1357.2,1384.2,920,1380.4,1380.4,1368.1,1377.2,1369.4,1358.1,True,True,True,False,9.2
2009.10.30,1383.7,1385.9,1368.6,1370.4,790,1384.7,1384.5,1378.7,1378.6,1373.0,1369.7,False,False,False,True,5.0
2009.10.31,1371.5,1383.5,1371.5,1382.0,512,1374.8,1374.8,1374.8,1383.5,1375.1,1372.3,False,False,False,False,3.3
2009.11.02,1379.2,1405.2,1379.2,1403.3,1144,1402.3,1401.6,1400.5,1390.5,1379.2,1379.8,True,True,True,True,21.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024.06.24,2354.2,2370.6,2353.0,2370.4,134044,2360.5,2359.9,2361.3,2358.4,2353.0,2354.6,True,True,True,True,7.1
2024.06.25,2369.5,2372.7,2350.5,2354.6,160721,2362.0,2361.3,2361.1,2372.6,2365.0,2367.7,False,False,False,False,8.4
2024.06.26,2354.5,2358.6,2327.6,2332.4,197254,2351.7,2351.2,2333.2,2347.1,2337.8,2352.3,True,True,True,True,21.3


In [17]:
def low_prob_MIN(df):
    counter = 0

    for i in range(len(df.index) - 2):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        poc = df.iloc[i]['MIN']

        if high >= poc >= low:
            counter = counter + 1

    return (counter / (len(df.index) - 1)) * 100

def low_prob_YL(df):
    counter = 0

    for i in range(len(df.index) - 2):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        yl = df.iloc[i]['low']

        if high >= yl >= low:
            counter = counter + 1

    return (counter / (len(df.index) - 1)) * 100

def low_prob_YH(df):
    counter = 0

    for i in range(len(df.index) - 2):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        yh = df.iloc[i]['high']

        if high >= yh >= low:
            counter = counter + 1

    return (counter / (len(df.index) - 1)) * 100

def low_prob_YO(df):
    counter = 0

    for i in range(len(df.index) - 2):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        yo = df.iloc[i]['open']

        if high >= yo >= low:
            counter = counter + 1

    return (counter / (len(df.index) - 1)) * 100


print("Porcentaje de test MIN Volume: ", low_prob_MIN(main_df))
print("Porcentaje de test YL: ", low_prob_YL(main_df))
print("Porcentaje de test YH: ", low_prob_YH(main_df))
print("Porcentaje de test YO: ", low_prob_YO(main_df))

Porcentaje de test MIN Volume:  51.45322434150772
Porcentaje de test YL:  45.43596730245232
Porcentaje de test YH:  49.52316076294278
Porcentaje de test YO:  51.20345140781109


In [18]:
#main_df.to_csv("OUTPUT.csv")

## Relación entre el Point Of Control (POC) y la zona de mayor transito de ticks (PHF).

In [19]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from datetime import timedelta

In [20]:
# connect to MetaTrader 5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

main_df = main_df.loc[start_dt:]
main_df['PHF'] = np.NaN # Create new column to allocate data
# iterate main_df for adding zone to data
for index, row in main_df.iterrows():
    dt1 = datetime.strptime(index.replace('.', '-'), "%Y-%m-%d")
    dt2 = dt1 + timedelta(hours=23, minutes=59, seconds=59) 
    
    ticks = mt5.copy_ticks_range(ticker, dt1, dt2, mt5.COPY_TICKS_ALL)
    # create DataFrame out of the obtained data
    df = pd.DataFrame(ticks)
    # convert time in seconds into the datetime format
    df['time']=pd.to_datetime(df['time'], unit='s')

    df = df.set_index('time')
    del df['last']
    del df['flags']
    del df['volume']
    del df['time_msc']
    del df['volume_real']

    flag_full_session = True #Flag for analising full day, not only the opening
    zone_output = 0 # Output from K-Means algorithm
    for index1, day in df.groupby(df.index.date):
        # Iterating each session
        start_session = pd.to_datetime(index1.strftime('%Y-%m-%d') + ' ' + '15:30:00')
        end_session = pd.to_datetime(index1.strftime('%Y-%m-%d') + ' ' + '16:00:00')

        # DF contains the session to analise.
        if flag_full_session:
            refdf = day
        else:
            refdf = day.loc[start_session:end_session]
        
        # create a MinMaxScaler object
        scaler = MinMaxScaler()

        # fit and transform the data
        normalized_data = scaler.fit_transform(refdf)

        # create a new DataFrame with the normalized data
        ndf = pd.DataFrame(normalized_data, columns=refdf.columns)

        ndf['ask'] = ndf['ask'].fillna(ndf['bid'])
        ndf['bid'] = ndf['bid'].fillna(ndf['ask'])

        ndf_to_matrix = ndf.values
        # Preparing data for clustering: Normalize time and price to have similar scales
        X_time = np.linspace(0, 1, len(ndf_to_matrix)).reshape(-1, 1)
        X_price = (ndf['ask'].values - np.min(ndf['ask'])) / (np.max(ndf['ask']) - np.min(ndf['ask']))
        X_cluster = np.column_stack((X_time, X_price))

        # Applying KMeans clustering
        num_clusters = 1
        kmeans = KMeans(n_clusters=num_clusters)
        kmeans.fit(ndf_to_matrix)

        # Extract cluster centers and rescale back to original price range
        cluster_centers = kmeans.cluster_centers_[:, 1] * (np.max(refdf['ask']) - np.min(refdf['ask'])) + np.min(refdf['ask'])
        
        zones = cluster_centers.tolist()
        #output = {
        #    "date": index1.strftime('%Y-%m-%d'),
        #    "zones": zones,
        #}
        zone_output = cluster_centers

    # Add output to main_df
    main_df.loc[index, 'PHF'] = zone_output

mt5.shutdown()
main_df

C:\Users\iamfr\AppData\Local\Temp\ipykernel_20920\3192721558.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['PHF'] = np.NaN # Create new column to allocate data
C:\Users\iamfr\AppData\Local\Temp\ipykernel_20920\3192721558.py:54: RuntimeWarning: invalid value encountered in divide
  X_price = (ndf['ask'].values - np.min(ndf['ask'])) / (np.max(ndf['ask']) - np.min(ndf['ask']))
C:\Users\iamfr\AppData\Local\Temp\ipykernel_20920\3192721558.py:54: RuntimeWarning: invalid value encountered in divide
  X_price = (ndf['ask'].values - np.min(ndf['ask'])) / (np.max(ndf['ask']) - np.min(ndf['ask']))
C:\Users\iamfr\AppData\Local\Temp\ipykernel_20920\3192721558.py:54: RuntimeWarning: invalid value encountered in divide
  X_price = (ndf['ask'].valu

,open,high,low,close,tick_volume,max_zone_high,max_zone_low,POC,min_zone_high,min_zone_low,MIN,SETUP_1,SETUP_2,SETUP_3,SETUP_4,SETUP_4_range,PHF
key_0,,,,,,,,,,,,,,,,,
2021.01.01,2141.4,2146.7,2135.2,2143.6,20251,2138.9,2138.6,2137.0,2146.0,2140.9,2141.3,False,False,False,False,4.4,0.000000
2021.01.04,2151.1,2197.1,2148.9,2191.9,209109,2192.9,2192.5,2159.1,2180.8,2162.1,2160.9,True,True,True,False,8.0,0.000000
2021.01.05,2191.9,2205.1,2182.5,2202.9,189796,2196.4,2196.2,2194.4,2189.6,2183.9,2186.7,True,True,True,True,2.5,0.000000
2021.01.06,2203.1,2212.4,2144.9,2153.0,308903,2204.2,2204.1,2177.4,2211.8,2204.3,2192.5,False,False,False,False,25.7,0.000000
2021.01.07,2153.0,2175.0,2146.9,2157.7,214247,2161.8,2161.4,2162.4,2174.2,2169.6,2173.2,True,True,True,True,9.4,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024.06.24,2354.2,2370.6,2353.0,2370.4,134044,2360.5,2359.9,2361.3,2358.4,2353.0,2354.6,True,True,True,True,7.1,2384.960106
2024.06.25,2369.5,2372.7,2350.5,2354.6,160721,2362.0,2361.3,2361.1,2372.6,2365.0,2367.7,False,False,False,False,8.4,2383.734566
2024.06.26,2354.5,2358.6,2327.6,2332.4,197254,2351.7,2351.2,2333.2,2347.1,2337.8,2352.3,True,True,True,True,21.3,2365.987845


In [21]:
main_df = main_df.drop(main_df[main_df.PHF <= 0.00].index)
main_df

,open,high,low,close,tick_volume,max_zone_high,max_zone_low,POC,min_zone_high,min_zone_low,MIN,SETUP_1,SETUP_2,SETUP_3,SETUP_4,SETUP_4_range,PHF
key_0,,,,,,,,,,,,,,,,,
2021.05.24,2116.3,2121.3,2107.6,2117.5,132253,2118.1,2117.8,2117.9,2112.9,2108.9,2111.7,True,True,True,True,1.6,1888.833628
2021.05.25,2117.5,2131.2,2104.8,2130.2,182663,2117.2,2116.9,2128.3,2112.9,2105.1,2108.9,False,False,False,False,10.8,1890.895585
2021.05.26,2130.2,2149.9,2129.9,2137.9,126136,2138.9,2138.5,2138.9,2138.0,2134.8,2134.0,True,True,True,True,8.7,1910.106489
2021.05.27,2137.9,2139.4,2122.0,2126.3,178792,2127.7,2127.4,2127.5,2139.1,2132.0,2137.9,True,True,True,True,10.4,1901.437029
2021.05.28,2126.4,2136.5,2114.7,2135.8,184323,2128.3,2127.9,2133.3,2125.7,2123.4,2122.3,False,False,False,False,6.9,1901.156451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024.06.24,2354.2,2370.6,2353.0,2370.4,134044,2360.5,2359.9,2361.3,2358.4,2353.0,2354.6,True,True,True,True,7.1,2384.960106
2024.06.25,2369.5,2372.7,2350.5,2354.6,160721,2362.0,2361.3,2361.1,2372.6,2365.0,2367.7,False,False,False,False,8.4,2383.734566
2024.06.26,2354.5,2358.6,2327.6,2332.4,197254,2351.7,2351.2,2333.2,2347.1,2337.8,2352.3,True,True,True,True,21.3,2365.987845


In [22]:
def num5_PHF_test(df):
    results = []

    for i in range(len(df.index) - 1):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        phf = df.iloc[i]['PHF']

        if apertura >= phf:
            # Bajista
            if phf <= high and phf >= low:
                results.append(True)
            else:
                results.append(False)

        elif apertura <= phf:
            # Alcista
            if phf <= high and phf >= low:
                results.append(True)
            else:
                results.append(False)
        else: 
            results.append(False)

    # Agregar un 0 adicional al final para igualar el tamaño de la columna con el dataframe original
    results.append(False)

    # Crear una nueva columna en el dataframe con los resultados
    df['SETUP_5'] = results

    return df['SETUP_5'].value_counts(normalize=True).mul(100).astype(str)+'%'

print("Porcentaje de test POC: ", num5_PHF_test(main_df))

Porcentaje de test POC:  SETUP_5
False     97.75840597758406%
True     2.2415940224159403%
Name: proportion, dtype: object


In [23]:
def cummulative_not_setup5_true(df):
    results = []
    tmp = 0

    for index, row in df.iterrows():
        if row['SETUP_4'] == False:
            tmp = tmp + 1
        else:
            results.append(tmp)
            tmp = 0

    nparr = np.array(results)

    return nparr.max()

print("Maximos fallos consecutivos de SETUP 5: ", cummulative_not_setup5_true(main_df))

Maximos fallos consecutivos de SETUP 5:  4


In [24]:
#Calculate crossed probability of SETUP 4 || 5
def cross_prob_setup4_OR_5(df):
    results = []
    tmp = 0

    for index, row in df.iterrows():
        if row['SETUP_4'] == True or row['SETUP_5'] == True:
            tmp = tmp + 1
    
    return (tmp/len(df))*100

#Calculate crossed probability of SETUP 4 && 5
def cross_prob_setup4_AND_5(df):
    results = []
    tmp = 0

    for index, row in df.iterrows():
        if row['SETUP_4'] == True and row['SETUP_5'] == True:
            tmp = tmp + 1
    
    return (tmp/len(df))*100

print("Probabilidad de SETUP 4 or SETUP 5: ", cross_prob_setup4_OR_5(main_df))
print("Probabilidad de SETUP 4 and SETUP 5: ", cross_prob_setup4_AND_5(main_df))

Probabilidad de SETUP 4 or SETUP 5:  72.47820672478207
Probabilidad de SETUP 4 and SETUP 5:  1.61892901618929


In [25]:
export_csv_df = main_df.copy()

del export_csv_df['close']
del export_csv_df['open']
del export_csv_df['high']
del export_csv_df['tick_volume']
del export_csv_df['max_zone_high']
del export_csv_df['max_zone_low']
del export_csv_df['low']
del export_csv_df['min_zone_high']
del export_csv_df['min_zone_low']
del export_csv_df['MIN']
del export_csv_df['SETUP_1']
del export_csv_df['SETUP_2']
del export_csv_df['SETUP_3']
del export_csv_df['SETUP_4']
del export_csv_df['SETUP_4_range']
del export_csv_df['SETUP_5']

#export_csv_df.to_csv('df_output.csv')						

## Backtesting

In [72]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.lib import SignalStrategy, TrailingStrategy

backtest_df = main_df.copy()

del backtest_df['max_zone_high']
del backtest_df['max_zone_low']
del backtest_df['min_zone_high']
del backtest_df['min_zone_low']
del backtest_df['MIN']
del backtest_df['SETUP_1']
del backtest_df['SETUP_2']
del backtest_df['SETUP_3']
del backtest_df['SETUP_4']
del backtest_df['SETUP_4_range']
del backtest_df['SETUP_5']

backtest_df.columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'POC', 'PHF']
backtest_df

,Open,High,Low,Close,Volume,POC,PHF
key_0,,,,,,,
2021.05.24,2116.3,2121.3,2107.6,2117.5,132253,2117.9,1888.833628
2021.05.25,2117.5,2131.2,2104.8,2130.2,182663,2128.3,1890.895585
2021.05.26,2130.2,2149.9,2129.9,2137.9,126136,2138.9,1910.106489
2021.05.27,2137.9,2139.4,2122.0,2126.3,178792,2127.5,1901.437029
2021.05.28,2126.4,2136.5,2114.7,2135.8,184323,2133.3,1901.156451
...,...,...,...,...,...,...,...
2024.06.24,2354.2,2370.6,2353.0,2370.4,134044,2361.3,2384.960106
2024.06.25,2369.5,2372.7,2350.5,2354.6,160721,2361.1,2383.734566
2024.06.26,2354.5,2358.6,2327.6,2332.4,197254,2333.2,2365.987845


In [54]:
m1, daily = data_from_csv()

In [57]:
tmp = start_dt+" 00:00:00"
cut = datetime.strptime(tmp, "%Y-%m-%d %H:%M:%S")
m1 = m1.loc[tmp:]
m1.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
m1

,Open,High,Low,Close,Volume
time,,,,,
2021-05-25 00:00:00,2117.5,2117.7,2117.4,2117.7,61
2021-05-25 00:01:00,2117.6,2117.6,2117.2,2117.2,72
2021-05-25 00:02:00,2117.3,2117.3,2117.0,2117.0,101
2021-05-25 00:03:00,2117.0,2117.1,2117.0,2117.0,48
2021-05-25 00:04:00,2117.0,2117.2,2116.9,2117.1,38
...,...,...,...,...,...
2024-06-28 21:55:00,2359.4,2359.6,2359.1,2359.3,130
2024-06-28 21:56:00,2359.2,2359.4,2359.1,2359.1,123
2024-06-28 21:57:00,2359.0,2359.2,2358.8,2359.1,107


In [85]:
class PHFStrategy(TrailingStrategy):
    tp_level = 0.01  # Take profit level as a percentage

    def init(self):
        super().init()
        self.set_trailing_sl(4)
        self.last_trade_date = None  # Track the date of the last trade to ensure only one trade per day

    def next(self):
        current_date = self.data.index[-1]  # Assuming the index of your data is a DatetimeIndex

        # Check if this is the first candle of the day (current date is different from last trade date)
        if self.last_trade_date != current_date:
            # Get the value from the "PHF" column for the first candle of the day
            try:
                phf_value = backtest_df['PHF'][current_date.strftime("%Y.%m.%d")] # Assuming 'PHF' is a column in your dataframe 
            except:
                pass
            # Make sure to place a trade based on the PHF value
            if not self.position:
                if phf_value > self.data.Open[-1]:  # Example condition: if PHF is positive, we buy
                    self.buy(tp=phf_value)
                    # Update the last trade date to the current date
                    self.last_trade_date = current_date
                else:  # Otherwise, we sell
                    self.sell(tp=phf_value)
                    # Update the last trade date to the current date
                    self.last_trade_date = current_date

# Assuming 'data' is your DataFrame with DateTime index and a 'PHF' column
# Here's how to run the backtest:

bt = Backtest(m1, PHFStrategy, cash=10_000, commission=.002)
stats = bt.run()
#bt.plot()
stats

Start                     2021-05-25 00:00:00
End                       2024-06-28 21:59:00
Duration                   1130 days 21:59:00
Exposure Time [%]                   99.999791
Equity Final [$]                  8543.247259
Equity Peak [$]                  11432.247259
Return [%]                         -14.567527
Buy & Hold Return [%]               11.370827
Return (Ann.) [%]                   -4.576271
Volatility (Ann.) [%]               14.037739
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -29.766676
Avg. Drawdown [%]                   -0.590198
Max. Drawdown Duration      603 days 08:18:00
Avg. Drawdown Duration        6 days 22:30:00
# Trades                                    2
Win Rate [%]                             50.0
Best Trade [%]                        10.5141
Worst Trade [%]                    -24.827089
Avg. Trade [%]                    

In [84]:
bt.plot()

c:\Python312\Lib\site-packages\backtesting\_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '4H'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "
c:\Python312\Lib\site-packages\backtesting\_plotting.py:126: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.resample(freq, label='right').agg(OHLCV_AGG).dropna()
c:\Python312\Lib\site-packages\backtesting\_plotting.py:136: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  equity_data = equity_data.resample(freq, label='right').agg(_EQUITY_AGG).dropna(how='all')
c:\Python312\Lib\site-packages\backtesting\_plotting.py:153: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  trades = trades.assign(count=1).resample(freq, on='ExitTime', label='right').agg(dict(
c:\Python31

ValueError: Length of values (2) does not match length of index (1)